# 导入库包配置GPU

In [1]:
# # Statistics tools
# from scipy import stats
# from scipy.stats import skew,norm
# from scipy.special import boxcox1p
# from scipy.stats import boxcox_normmax
# import scipy.stats as stats

# Visualizing tools
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling as ppf
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# # 设置中文和'-'负号显示问题
# from pylab import mpl
# mpl.rcParams['font.sans-serif'] = ['FangSong']  # 设置matplotlib可以显示汉语
# mpl.rcParams['axes.unicode_minus'] = True

# Preprocessing tools
import math
import time
import numpy as np
import pandas as pd

# DL/ML Algoirthm
import torch
from torch import nn
from torch.utils import data as tud
from torchkeras import summary
from d2l import torch as d2l


import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import warnings
warnings.filterwarnings(action="ignore")

gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if gpu else "cpu")
gpu, device

(True, device(type='cuda', index=0))

In [3]:
!pip install tsai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/4a/7d/d352ae8f0aa2170f9e0ae4676148675a738cf9fd0c034bd024b82f7df8ed/fastdownload-0.0.5-py3-none-any.whl (13 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c3/73/fec9fad97b97e879338f1e30fd1717d7a802ce702df8837649349440f2e6/statsmodels-0.13.2-cp39-cp39-win_amd64.whl (9.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/47/d8/41f71e08248b487e90eb6b5823077e93a8991be92b026b9727d4478e83af/numpy-1.19.3-cp39-cp39-win_amd64.whl (13.3 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/87/7f/d37cd027c25145eeba92b1a756976931c831803d92547c8637a3400c339f/patsy-0.5.2-py2.py3-none-any.whl (233 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/07/3c/bf72ebd3e78eb1ef773f4f0650ecdc29c6454aeafe9c08f6da3f227dd2bc/cloudpickle-2.0.0-py3-none-

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Huris\anaconda3\envs\sEMG\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Huris\\AppData\\Local\\Temp\\pip-install-lp30ma6k\\sktime_8c90ed55eec343afa3bc2137807b90bb\\setup.py'"'"'; __file__='"'"'C:\\Users\\Huris\\AppData\\Local\\Temp\\pip-install-lp30ma6k\\sktime_8c90ed55eec343afa3bc2137807b90bb\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Huris\AppData\Local\Temp\pip-wheel-zqvxjmij'
       cwd: C:\Users\Huris\AppData\Local\Temp\pip-install-lp30ma6k\sktime_8c90ed55eec343afa3bc2137807b90bb\
  Complete output (21 lines):
  Traceback (most recent call last):
    File "C:\Users\Huris\AppData\Local\Temp\pip-install-lp30ma6k\sktim

# 数据导入

In [26]:
cols = ["One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Rect", "Time_ns"]

train1_data = pd.read_csv("DLdata/Train1.csv", header=None)
train2_data = pd.read_csv("DLdata/Train2.csv", header=None)
train3_data = pd.read_csv("DLdata/Train3.csv", header=None)
train_data = pd.read_csv("DLdata/Train.csv", header=None)

test_data = pd.read_csv("DLdata/Test.csv", header=None)
valid_data = pd.read_csv("DLdata/Valid.csv", header=None)

train1_data.columns = train2_data.columns = train3_data.columns = cols
train_data.columns = test_data.columns = valid_data.columns = cols

- train_data.shape：查看数据的形状
- train_data.head()：查看数据本身
- train_data.info()：数据集的简单描述， 总行数，空值或者数据类型等
- train_data.value_counts()：分类的时候用的多，查看类别的个数
- train_data.describe()：简单的统计性表述，最大值，最小，平均等
- train_data.corr(method='pearson')：查看列之间的相关性
- train_data.skew()：通过分析数据的高斯分布来确认数据的偏离情况

In [27]:
# 做分类处理
# 0->0，1->0，2->0，8->0，9->0
# 3->5，4->5，5->5，6->5，7->5

def RoundClassification(OriginData):
    Index = []
    # 1->0
    Index = OriginData % 10 == 1
    OriginData[Index] = OriginData[Index] - 1
    # 2->0
    Index = OriginData % 10 == 2
    OriginData[Index] = OriginData[Index] - 2
    # 8->0
    Index = OriginData % 10 == 8
    OriginData[Index] = OriginData[Index] + 2
    # 9->0
    Index = OriginData % 10 == 9
    OriginData[Index] = OriginData[Index] + 1
    # 3->5
    Index = OriginData % 10 == 3
    OriginData[Index] = OriginData[Index] + 2
    # 4->5
    Index = OriginData % 10 == 4
    OriginData[Index] = OriginData[Index] + 1
    # 6->5
    Index = OriginData % 10 == 6
    OriginData[Index] = OriginData[Index] - 1
    # 7->5
    Index = OriginData % 10 == 7
    OriginData[Index] = OriginData[Index] - 2

    
RoundClassification(train1_data.Rect)
RoundClassification(train2_data.Rect)
RoundClassification(train3_data.Rect)
RoundClassification(train_data.Rect)

RoundClassification(test_data.Rect)
RoundClassification(valid_data.Rect)

train_data, test_data, valid_data

(      One  Two  Three  Four  Five  Six  Seven  Eight  Rect      Time_ns
 0      84  268    736   161    57  285     76    209    20    332902500
 1      90  262    717   147    54  292     78    216    20    345693900
 2      96  291    708   173    61  297     80    205    20    378956900
 3      96  361    719   164    58  302     80    208    20    399868600
 4      85  374    746   148    55  309     75    215    25    421466300
 ...   ...  ...    ...   ...   ...  ...    ...    ...   ...          ...
 8926  212  596    640   186    65  573    360    488    30  59938979500
 8927  215  588    647   189    69  588    434    561    30  59958747600
 8928  194  589    653   185    60  601    477    573    30  59979173500
 8929  200  464    649   160    53  629    499    579    30  59997057800
 8930  197  438    629   143    53  661    508    606    35  60017252700
 
 [8931 rows x 10 columns],
       One  Two  Three  Four  Five  Six  Seven  Eight  Rect      Time_ns
 0     108  350    736

In [39]:
train_features = torch.tensor(train_data.drop(['Rect', 'Time_ns'], axis=1).values, dtype=torch.float32)
train_labels = torch.tensor(train_data.Rect.values.reshape(-1, 1), dtype=torch.float32)
train_six_features = torch.tensor(train_data.drop(['Two', 'Five', 'Rect', 'Time_ns'], axis=1).values, dtype=torch.float32)
train_two_features = torch.cat((train_six_features[:,1].reshape(-1, 1), train_six_features[:,2].reshape(-1, 1)), dim = 1)

test_features = torch.tensor(test_data.drop(['Rect', 'Time_ns'], axis=1).values, dtype=torch.float32)
test_labels = torch.tensor(test_data.Rect.values.reshape(-1, 1), dtype=torch.float32)
test_six_features = torch.tensor(test_data.drop(['Two', 'Five', 'Rect', 'Time_ns'], axis=1).values, dtype=torch.float32)
test_two_features = torch.cat((test_six_features[:,1].reshape(-1, 1), test_six_features[:,2].reshape(-1, 1)), dim = 1)

valid_features = torch.tensor(valid_data.drop(['Rect', 'Time_ns'], axis=1).values, dtype=torch.float32)
valid_labels = torch.tensor(valid_data.Rect.values.reshape(-1, 1), dtype=torch.float32)
valid_six_features = torch.tensor(valid_data.drop(['Two', 'Five', 'Rect', 'Time_ns'], axis=1).values, dtype=torch.float32)
valid_two_features = torch.cat((valid_six_features[:,1].reshape(-1, 1), valid_six_features[:,2].reshape(-1, 1)), dim = 1)

# train_features.shape, train_labels.shape, test_features.shape, test_labels.shape, valid_features.shape, valid_labels.shape

(tensor([[ 84., 736., 161., 285.,  76., 209.],
         [ 90., 717., 147., 292.,  78., 216.],
         [ 96., 708., 173., 297.,  80., 205.],
         ...,
         [194., 653., 185., 601., 477., 573.],
         [200., 649., 160., 629., 499., 579.],
         [197., 629., 143., 661., 508., 606.]]),
 tensor([[736., 161.],
         [717., 147.],
         [708., 173.],
         ...,
         [653., 185.],
         [649., 160.],
         [629., 143.]]),
 tensor([[108., 736., 189., 337., 182., 331.],
         [116., 733., 201., 339., 185., 343.],
         [134., 797., 241., 350., 202., 337.],
         ...,
         [170., 528., 151., 502., 622., 611.],
         [172., 517., 154., 411., 436., 524.],
         [191., 402., 178., 386., 533., 559.]]),
 tensor([[736., 189.],
         [733., 201.],
         [797., 241.],
         ...,
         [528., 151.],
         [517., 154.],
         [402., 178.]]),
 tensor([[100., 812., 151., 272., 180., 222.],
         [109., 833., 154., 272., 173., 231.],
  

In [8]:
# # 展示数据报告
# ppf.ProfileReport(train_data)

# 数据预处理

In [42]:
train_DataLoader = tud.DataLoader(tud.TensorDataset(train_features, train_labels) , batch_size=64, shuffle=True, num_workers=2, drop_last=True)

# 测试数据，发现有点问题，前面两组数据始终为0，因此加了一句判断
for features, labels in train_DataLoader:
    print(features,labels)

tensor([[ 401.,  944., 1037.,  662.,  271.,  399.,  143.,  191.],
        [ 146.,  331.,  267.,   61.,   60.,  776.,  551.,  724.],
        [ 121.,  356.,  697.,  117.,   51.,  401.,  108.,   43.],
        [ 292.,  564.,  266.,   68.,   90.,  799.,  574., 1046.],
        [ 610., 1141.,  467.,  125.,  235., 1319.,  929.,  938.],
        [ 328.,  787.,  128.,   65.,  108., 1295.,  881.,  889.],
        [ 114.,  439., 1050.,  246.,   91.,  274.,  108.,   55.],
        [  71.,  300.,  949.,  114.,   54.,  105.,   53.,   33.],
        [  95.,  397.,  754.,  183.,   71.,  375.,  163.,  225.],
        [ 202.,  579.,  656.,  113.,   88.,  976.,  737.,  837.],
        [ 384.,  912.,  184.,   81.,  113., 1183., 1004.,  798.],
        [ 223.,  638.,  896.,  325.,  170.,  696.,  312.,  718.],
        [ 275.,  630.,  618.,  314.,  150.,  775.,  543.,  763.],
        [ 136.,  394.,  874.,  150.,   67.,  192.,   86.,  117.],
        [ 205.,  494.,  798.,  189.,   93.,  839.,  473.,  833.],
        [ 

        [ 181.,  579.,  628.,  169.,   86.,  612.,  512.,  702.]]) tensor([[70.],
        [10.],
        [80.],
        [60.],
        [20.],
        [90.],
        [ 5.],
        [10.],
        [80.],
        [65.],
        [15.],
        [10.],
        [35.],
        [30.],
        [25.],
        [45.],
        [10.],
        [65.],
        [45.],
        [95.],
        [30.],
        [30.],
        [65.],
        [15.],
        [60.],
        [20.],
        [75.],
        [35.],
        [40.],
        [30.],
        [90.],
        [40.],
        [10.],
        [65.],
        [60.],
        [30.],
        [70.],
        [25.],
        [35.],
        [ 0.],
        [30.],
        [95.],
        [85.],
        [30.],
        [80.],
        [75.],
        [ 5.],
        [75.],
        [10.],
        [60.],
        [45.],
        [50.],
        [80.],
        [55.],
        [65.],
        [15.],
        [ 0.],
        [30.],
        [10.],
        [55.],
        [ 0.],
        [30.],
   

        [ 121.,  458.,  233.,   44.,   38.,  321.,  561.,  523.]]) tensor([[ 20.],
        [ 25.],
        [ 70.],
        [ 55.],
        [ 85.],
        [ 15.],
        [ 70.],
        [ 70.],
        [ 85.],
        [ 65.],
        [ 25.],
        [ 60.],
        [ 70.],
        [ 40.],
        [ 75.],
        [ 95.],
        [ 25.],
        [ 35.],
        [ 55.],
        [ 85.],
        [ 15.],
        [ 25.],
        [ 80.],
        [ 10.],
        [ 55.],
        [ 85.],
        [ 20.],
        [ 75.],
        [ 40.],
        [ 30.],
        [ 50.],
        [ 70.],
        [ 60.],
        [ 10.],
        [ 35.],
        [100.],
        [ 15.],
        [ 70.],
        [ 55.],
        [ 60.],
        [ 60.],
        [ 50.],
        [  5.],
        [ 15.],
        [ 85.],
        [ 40.],
        [ 65.],
        [ 10.],
        [ 15.],
        [  0.],
        [ 95.],
        [ 55.],
        [ 10.],
        [ 80.],
        [ 45.],
        [ 20.],
        [ 30.],
        [ 75.],
     

        [  86.,  216.,  840.,  229.,   52.,  461.,  111.,   68.]]) tensor([[90.],
        [45.],
        [35.],
        [25.],
        [95.],
        [15.],
        [80.],
        [20.],
        [45.],
        [40.],
        [35.],
        [40.],
        [75.],
        [30.],
        [45.],
        [30.],
        [45.],
        [80.],
        [50.],
        [90.],
        [30.],
        [85.],
        [95.],
        [40.],
        [85.],
        [70.],
        [20.],
        [25.],
        [75.],
        [ 0.],
        [35.],
        [15.],
        [45.],
        [35.],
        [45.],
        [40.],
        [15.],
        [30.],
        [45.],
        [65.],
        [70.],
        [60.],
        [85.],
        [70.],
        [50.],
        [40.],
        [80.],
        [ 0.],
        [65.],
        [ 0.],
        [70.],
        [95.],
        [70.],
        [80.],
        [15.],
        [60.],
        [35.],
        [85.],
        [25.],
        [35.],
        [10.],
        [85.],
   

tensor([[ 114.,  566., 1164.,  263.,  105.,  348.,  129.,   54.],
        [ 142.,  401.,  454.,   94.,   60.,  488.,  551.,  668.],
        [ 122.,  473.,  332.,   66.,   40.,  467.,  336.,  567.],
        [ 558.,  975.,  461.,  185.,  271., 1490., 1083., 1090.],
        [ 160.,  486.,  482.,  129.,   63.,  366.,  348.,  613.],
        [ 189.,  457.,  343.,   79.,   53.,  485.,  462.,  654.],
        [  71.,  368.,  975.,  155.,   56.,  182.,   65.,   36.],
        [ 160.,  588.,  998.,  377.,  131.,  524.,  124.,   92.],
        [ 194.,  491.,  618.,  151.,   84.,  831.,  440.,  468.],
        [ 206.,  489.,  175.,   61.,   84.,  970.,  700.,  832.],
        [ 106.,  342.,  415.,  102.,   45.,  272.,  365.,  553.],
        [ 276.,  808.,  233.,   57.,   92.,  996.,  720.,  902.],
        [ 542.,  929.,  413.,  163.,  207., 1272.,  919., 1197.],
        [ 225.,  605.,  182.,   61.,   87.,  959.,  695.,  876.],
        [ 182.,  458.,  642.,  179.,   48.,  548.,  212.,   77.],
        [ 

# 模型建立

In [8]:
# Encoder Decoder Architecture + ResNet

class HbConNet(nn.Module):
    def __init__(self):
        super(HbConNetb, self).__init__()
        
        
    
    def forward(self, x):
        
        return x


    